In [1]:
!pip install gym

tensorflow 1.12.0 has requirement keras-applications>=1.0.6, but you'll have keras-applications 1.0.2 which is incompatible.
tensorflow 1.12.0 has requirement keras-preprocessing>=1.0.5, but you'll have keras-preprocessing 1.0.1 which is incompatible.
You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import gym
import random
import numpy as np

In [3]:
# in this game action is discrete either 0 or 1
env_name =  'CartPole-v1'
env = gym.make(env_name)

In [4]:
print(env.observation_space)

print(env.action_space)

print(env.action_space.n)

Box(4,)
Discrete(2)
2


In [5]:
state = env.reset()  # initial position

for _ in range(200):
    
    
#     action = env.action_space.sample()  # choose random action
    pole_angle = state[2]
    action = 0 if pole_angle < 0 else 1
    state , reward , done , info  = env.step(action)
    env.render()  # to show
    
env.close()

# we can also use action_size = env.action_space.n
# action = random.choice(range(action_size))

WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.


In [6]:
env_name =  'MountainCarContinuous-v0'
env = gym.make(env_name)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [7]:
print(env.observation_space)

Box(2,)


In [8]:
print(env.action_space)

Box(1,)


In [9]:
print(env.action_space.low)

[-1.]


In [10]:
print(env.action_space.high)

[1.]


In [11]:
print(env.action_space.shape)

(1,)


In [12]:
state = env.reset()

for _ in range(200):
    action = env.action_space.sample()
    state , reward , done , info = env.step(action)
    env.render()
    
env.close()

# Reinforcement Learning (Q-Learning)

In [13]:
# observtions are quantities to measure state 
# Q (s(t) , a(t)) = r(t+1) + gamma r(t+2) + (gamma)^2 r(t+3)---+---------
# where gamma is discount factor
# Q(s(t)  , a(t)) = r(t+1) + gamma . Q(s(t+1) , a(t+1))

# Q(st , at) = rt + 1 + G.Qmax(st + 1) 

In [14]:
import gym
import random 
import numpy as np
import time
from IPython.display import clear_output

In [15]:
from gym.envs.registration import register
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4','is_slippery' : False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass
env_name = 'FrozenLakeNoSlip-v0'
env = gym.make(env_name)

In [16]:
print(env.observation_space)

Discrete(16)


In [17]:
print(env.action_space)

Discrete(4)


In [18]:
type(env.action_space)

gym.spaces.discrete.Discrete

In [19]:
class Agent():   # agent is player , class is blueprint ie template
    def __init__(self,env):  # self is this
        self.is_discrete = type(env.action_space) == gym.spaces.discrete.Discrete
        if self.is_discrete:
            self.action_size = env.action_space.n
        else:
            self.action_low = self.action_space.low
            self.action_high = self.action_space.high
            self.action_space = env.action_space.shape
            
    def get_action(self,state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
            
        else:
            action = np.random.uniform(self.action_low,
                                      self.action_high,
                                      self.action_shape)
            
        return action

In [20]:
agent = Agent(env)

In [21]:
for ep in range(2):
    

    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state , reward , done , info = env.step(action)

        env.render()
        time.sleep(0.05)
        clear_output(wait=True)


  (Down)
SFFF
FHFH
FFFH
HFFG


In [22]:
class QAgent(Agent):
    
    
    def __init__(self,env,discount_rate = 0.97 , learning_rate = 0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        self.exploration_rate = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.q_table = 1e-4 * np.random.random([self.state_size,self.action_size])
        
        
    def get_action(self,state):
        q_state = self.q_table[state]
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.exploration_rate else action_greedy
    
    
    def train(self,experience):
        state , action , next_state , reward , done = experience
        
        q_next = self.q_table[next_state]
        q_next = np.zeros([self.action_size]) if done else q_next
        q_target = reward + self.discount_rate * np.max(q_next)
        
        q_update = q_target - self.q_table[state,action]  # gradient descend , mutlipy by learning rate
        self.q_table[state , action] += self.learning_rate * q_update
        
        if done:
            self.exploration_rate *= 0.99 # it reduces every_time
        
        
    

In [23]:
agent = QAgent(env)

In [24]:
total_reward = 0

for ep in range(100):
    

    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state , reward , done , info = env.step(action)
        agent.train((state , action , next_state , reward , done))
        state = next_state
        total_reward += reward
        print(f"s:{state} a:{action}")
        print(f"Episode:{ep} Reward: {total_reward} Explore: {agent.exploration_rate}")

        env.render()
        
        print(agent.q_table)
        
        time.sleep(0.05)
        clear_output(wait=True)


s:5 a:2
Episode:99 Reward: 5.0 Explore: 0.36603234127322926
  (Right)
SFFF
FHFH
FFFH
HFFG
[[8.92725955e-05 6.92407727e-05 6.40747682e-05 7.30372916e-05]
 [1.73014496e-05 3.93695995e-05 4.35847721e-05 8.93770999e-05]
 [6.22673496e-05 1.34198721e-05 8.72495265e-05 1.16142250e-05]
 [9.52943169e-05 3.98775877e-05 1.42502668e-05 2.20608527e-05]
 [3.12350367e-05 3.97879046e-05 4.01602690e-05 2.49958230e-05]
 [4.75156285e-06 7.05590341e-05 7.47561780e-05 6.14505788e-05]
 [1.92675813e-05 8.94852162e-05 1.06352740e-05 9.16149307e-06]
 [1.55196901e-05 3.75067691e-05 6.45829488e-05 7.71718893e-05]
 [3.28185092e-05 7.74131902e-05 4.29415561e-06 4.73307514e-05]
 [6.15016135e-05 1.07935745e-05 7.09837774e-06 6.02893505e-05]
 [9.72480686e-05 1.62452296e-03 4.39235385e-05 8.14230864e-05]
 [6.91267787e-05 9.84427516e-05 1.79932281e-05 4.55205886e-05]
 [8.50467470e-05 8.29483742e-05 7.13601934e-05 8.63319819e-05]
 [8.80267515e-05 7.32638259e-05 7.61381231e-06 1.82176029e-05]
 [4.92873677e-05 1.01429935e

# Q learning using nueral nets

In [25]:
import tensorflow as tf

In [31]:
class QNAgent(Agent):
    def __init__(self , env , discount_rate = 0.97 , learning_rate = 0.001):
        super().__init__(env)
        self.state_size = env.observation_space.n
        self.exploration_rate = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        
        self.build_model()
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
        
    def build_model(self):
        
        tf.reset_default_graph()
        
        self.state_in = tf.placeholder(tf.int32 , shape = [1])   # for single row
        self.action_in = tf.placeholder(tf.int32 , shape = [1])
        self.target_in = tf.placeholder(tf.float32 , shape = [1])
        
        self.state = tf.one_hot(self.state_in , depth = self.state_size)
        self.action = tf.one_hot(self.action_in , depth = self.action_size)
        
        self.q_state = tf.layers.dense(self.state,units = self.action_size , name = 'q_table')
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state , self.action) , axis = 1)  
        
        self.loss = tf.reduce_sum(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
        
    def get_action(self,state):
        
        q_state = self.sess.run(self.q_state,feed_dict = {self.state_in : [state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.exploration_rate else action_greedy
    
    def train(self , experience):
        state , action , next_state , reward , done = [[exp] for exp in  experience]
        
        q_next = self.sess.run(self.q_state , feed_dict = {self.state_in: next_state})
        q_next[done] = np.zeros(self.action_size)
        
        q_target = reward + self.discount_rate * np.max(q_next)
        feed = {self.state_in : state , self.action_in:action , self.target_in:q_target} 
        feed = self.sess.run(self.optimizer , feed_dict=feed)
        
        if done[0]: # in dict
            self.exploration_rate *= 0.99
            
            
    def __del__(self):
        self.sess.close()
        
        
        
        

In [32]:
agent = QNAgent(env)

In [38]:
total_reward = 0.0
for ep in range(100):
    

    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state , reward , done , info = env.step(action)
        agent.train((state , action , next_state , reward , done))
        state = next_state
        total_reward += reward
        

        print(f"s:{state} a:{action}")
        print(f"Episode:{ep} Reward: {total_reward} Explore: {agent.exploration_rate}")

        env.render()
        
        with tf.variable_scope('q_table' , reuse = True):
            weights = agent.sess.run(tf.get_variable('kernel'))
            print(weights)
        
        time.sleep(0.05)
        clear_output(wait=True)


s:5 a:2
Episode:99 Reward: 3.0 Explore: 0.04904089407128576
  (Right)
SFFF
FHFH
FFFH
HFFG
[[ 0.09488706  1.2385808   0.73185223  0.9064963 ]
 [ 0.9520302   1.6178434   0.45488712 -0.2464204 ]
 [ 1.4635905   1.40746    -0.37101513 -1.2244873 ]
 [ 2.1063316   0.6461236  -0.00802113 -0.3023706 ]
 [-0.25106683 -0.4039877   0.98839456 -0.10738462]
 [-0.09967619 -0.22012347  0.30218738  0.05771732]
 [-1.2502596   0.8404912  -0.18106368  0.07739478]
 [-0.22580075 -0.19202867 -0.07839805  0.45591414]
 [-0.11133487 -1.9464959   1.2561946  -0.01497089]
 [ 1.1164619  -0.14226075  0.61659557 -0.55261505]
 [ 0.5494346   0.9701312  -2.1539297   0.2690145 ]
 [ 0.37470508  0.1430834   0.17389297 -0.24357486]
 [-0.4857506  -0.35783893  0.5065137   0.37363505]
 [-1.0324613  -0.3846952  -0.3108703   1.5437584 ]
 [ 0.14072025 -0.12420323 -0.42261964  0.5294349 ]
 [-0.4171778  -0.03310412  0.21447754  0.42376786]]
